In [1]:
###### Set Up #####
# verify our folder with the data and module assets is installed
# if it is installed make sure it is the latest
!test -e ds-assets && cd ds-assets && git pull && cd ..
# if it is not installed clone it 
!test ! -e ds-assets && git clone https://github.com/lutzhamel/ds-assets.git
# point to the folder with the assets
home = "ds-assets/assets/" 
import sys
sys.path.append(home)      # add home folder to module search path

Cloning into 'ds-assets'...
remote: Enumerating objects: 74, done.
remote: Counting objects: 100% (74/74), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 74 (delta 13), reused 70 (delta 11), pack-reused 0
Unpacking objects: 100% (74/74), done.


# Data Manipulation with Pandas

Pandas supports 1-D (Series), 2-D (DataFrame), and 3-D (Panel) data structures.  Here we cover DataFrames because they most closely resemble the kind of data tables data scientists mostly look at.

The advantage of Pandas is that it stores the data together with its *metadata*.

The most often used meta data with Pandas are the **column names** and the **index**.


In [2]:
import pandas
import numpy # for random number generation

In [3]:
df = pandas.read_csv(home+"mammals.csv")

In [4]:
df

,Legs,Wings,Fur,Feathers,Mammal
0,4,no,yes,no,True
1,2,yes,no,yes,False
2,4,no,no,no,False
3,4,yes,yes,no,True
4,3,no,no,no,False


# DataFrame Parts

A dataframe is composed of different parts that work together to give a coherent view of the data:

In [5]:
df.columns

Index(['Legs', 'Wings', 'Fur', 'Feathers', 'Mammal'], dtype='object')

In [6]:
df.index

RangeIndex(start=0, stop=5, step=1)

In [7]:
df.values

array([[4, 'no', 'yes', 'no', True],
       [2, 'yes', 'no', 'yes', False],
       [4, 'no', 'no', 'no', False],
       [4, 'yes', 'yes', 'no', True],
       [3, 'no', 'no', 'no', False]], dtype=object)

We can change the parts of the data.  For example, we can create a new index for our dataframe:

In [8]:
df.index = ['Dog', 'Duck', 'Frog', 'Bat', 'Bar Stool']

In [9]:
df

,Legs,Wings,Fur,Feathers,Mammal
Dog,4,no,yes,no,True
Duck,2,yes,no,yes,False
Frog,4,no,no,no,False
Bat,4,yes,yes,no,True
Bar Stool,3,no,no,no,False


# Indexing and Slicing

For array-style indexing Pandas  uses the **loc**, **iloc**, and **ix** indexers. 

Using the **iloc** indexer, we can index the underlying array as if it is a simple array using row and column integer values (hence the i in iloc). The DataFrame index and column labels are maintained in the result:

In [10]:
df

,Legs,Wings,Fur,Feathers,Mammal
Dog,4,no,yes,no,True
Duck,2,yes,no,yes,False
Frog,4,no,no,no,False
Bat,4,yes,yes,no,True
Bar Stool,3,no,no,no,False


In [11]:
df.iloc[:2,1:4]

,Wings,Fur,Feathers
Dog,no,yes,no
Duck,yes,no,yes


Using the **loc** indexer we can index the underlying data in an array-like style but using the explicit index and column names:

In [12]:
df.loc[:'Duck','Wings':'Feathers']

,Wings,Fur,Feathers
Dog,no,yes,no
Duck,yes,no,yes


Notice that when slicing with an explicit index (i.e., data.loc['a':'c']), the final index is included in the slice, while when slicing with an implicit index (i.e., data.iloc[0:2]), the final index is excluded from the slice.

The indexer **ix** allows the mix of integer and explicit indexing.

# Data Access Patterns

We can use relational and boolean expressions when selecting data from a dataframe.

In order to see that we have to realize that there is another simple way to select frame columns:

In [13]:
df[['Wings', 'Mammal']] # using a list of column names to access columns

,Wings,Mammal
Dog,no,True
Duck,yes,False
Frog,no,False
Bat,yes,True
Bar Stool,no,False


Relational Operators:

In [14]:
df[df.Wings == 'yes'] # accessing rows for which the equality holds

,Legs,Wings,Fur,Feathers,Mammal
Duck,2,yes,no,yes,False
Bat,4,yes,yes,no,True


In [15]:
df[df.Wings == 'yes'].Mammal # accessing attribute values for rows for which the equality holds

Duck    False
Bat      True
Name: Mammal, dtype: bool

In [16]:
df[(df.Wings == 'yes') & (df.Fur == 'yes')] # boolean operations

,Legs,Wings,Fur,Feathers,Mammal
Bat,4,yes,yes,no,True


# Filtering

Boolean indexing using a Boolean

In [17]:
df.Mammal

Dog           True
Duck         False
Frog         False
Bat           True
Bar Stool    False
Name: Mammal, dtype: bool

In [18]:
df.Mammal.dtype

dtype('bool')

In [19]:
df[df.Mammal == True] # or we could have just said df[df.Mammal]

,Legs,Wings,Fur,Feathers,Mammal
Dog,4,no,yes,no,True
Bat,4,yes,yes,no,True


Filtering using **isin()**

In [20]:
df.index.isin(['Dog','Bat'])

array([ True, False, False,  True, False])

Any Boolean vector can be used as filter.

In [21]:
v = df.index.isin(['Dog','Bat'])
df[v]

,Legs,Wings,Fur,Feathers,Mammal
Dog,4,no,yes,no,True
Bat,4,yes,yes,no,True


# Combining DataFrames

Using concat() along axis 1 (columns)
 

In [22]:
df1 = df.iloc[:,:2]
df1

,Legs,Wings
Dog,4,no
Duck,2,yes
Frog,4,no
Bat,4,yes
Bar Stool,3,no


In [23]:
df2 = df.iloc[:,2:]
df2

,Fur,Feathers,Mammal
Dog,yes,no,True
Duck,no,yes,False
Frog,no,no,False
Bat,yes,no,True
Bar Stool,no,no,False


In [24]:
pandas.concat([df1,df2],axis=1)

,Legs,Wings,Fur,Feathers,Mammal
Dog,4,no,yes,no,True
Duck,2,yes,no,yes,False
Frog,4,no,no,no,False
Bat,4,yes,yes,no,True
Bar Stool,3,no,no,no,False


**Note** The two dataframes have to agree on the index!

Creating a new index on df2:

In [25]:
df2.reset_index(drop=True, inplace=True)
df2

,Fur,Feathers,Mammal
0,yes,no,True
1,no,yes,False
2,no,no,False
3,yes,no,True
4,no,no,False


In [26]:
pandas.concat([df1,df2],axis=1)

,Legs,Wings,Fur,Feathers,Mammal
0,NaN,NaN,yes,no,True
1,NaN,NaN,no,yes,False
2,NaN,NaN,no,no,False
3,NaN,NaN,yes,no,True
4,NaN,NaN,no,no,False
Bar Stool,3.0,no,NaN,NaN,NaN
Bat,4.0,yes,NaN,NaN,NaN
Dog,4.0,no,NaN,NaN,NaN
Duck,2.0,yes,NaN,NaN,NaN
Frog,4.0,no,NaN,NaN,NaN


Using concat() along axis 0 (rows)


In [27]:
df_row_1 = df.iloc[:2,:]
df_row_1

,Legs,Wings,Fur,Feathers,Mammal
Dog,4,no,yes,no,True
Duck,2,yes,no,yes,False


In [28]:
df_row_2 = df.iloc[2:,:]
df_row_2

,Legs,Wings,Fur,Feathers,Mammal
Frog,4,no,no,no,False
Bat,4,yes,yes,no,True
Bar Stool,3,no,no,no,False


In [29]:
pandas.concat([df_row_1,df_row_2],axis=0)

,Legs,Wings,Fur,Feathers,Mammal
Dog,4,no,yes,no,True
Duck,2,yes,no,yes,False
Frog,4,no,no,no,False
Bat,4,yes,yes,no,True
Bar Stool,3,no,no,no,False


**Note** dataframes have to agree on column names!

**append** and **assign** functions work very similarly.

# Missing or Duplicated Data
* Pandas flags missing values with NaN (not a number).
* In most cases, any computations applied to a dataframe with NaNs will ignore the NaNs
* However, it is still a good idea to clean up the dataframe
* In general we have two options to deal with missing data:
 * Either drop the row or columns that has NaNs
 * Or try to substitute a reasonable value for the NaN
 

Generate a dataset with NaNs

In [30]:
df = pandas.DataFrame(numpy.random.randn(4, 3), index=['a', 'c', 'd', 'e'], 
                  columns=['one', 'two', 'three'])
df

,one,two,three
a,3.060336,-2.625102,1.193727
c,-1.144262,1.758824,-0.205974
d,1.363693,-0.125507,0.338207
e,-1.228161,-1.154391,1.012711


Generating NaNs

In [31]:
df2 = df.reindex(['a', 'b', 'c', 'd', 'e'])
df2

,one,two,three
a,3.060336,-2.625102,1.193727
b,NaN,NaN,NaN
c,-1.144262,1.758824,-0.205974
d,1.363693,-0.125507,0.338207
e,-1.228161,-1.154391,1.012711


In [32]:
# find the places where the NaNs are
df2.isnull()

,one,two,three
a,False,False,False
b,True,True,True
c,False,False,False
d,False,False,False
e,False,False,False


In [33]:
# look at the values of the isnull dataframe
df2.isnull().values

array([[False, False, False],
       [ True,  True,  True],
       [False, False, False],
       [False, False, False],
       [False, False, False]])

In [34]:
# find out how many values are missing
# NOTE: sum treats 'True' as 1 and 'False' as 0 
df2.isnull().values.sum()

3

In [35]:
# drop rows that have NaNs
df2.dropna(how='any',axis=0)

,one,two,three
a,3.060336,-2.625102,1.193727
c,-1.144262,1.758824,-0.205974
d,1.363693,-0.125507,0.338207
e,-1.228161,-1.154391,1.012711


In [36]:
# dropping columns that have NaNs
# NOTE: this is NOT always a good idea -- empty dataframe!
df2.dropna(how='any',axis=1)

""
a
b
c
d
e


# Replacing Missing Data

We can also try to estimate the missing data - **impute** it.

We replace the missing values by the means of each column.

In [37]:
df2

,one,two,three
a,3.060336,-2.625102,1.193727
b,NaN,NaN,NaN
c,-1.144262,1.758824,-0.205974
d,1.363693,-0.125507,0.338207
e,-1.228161,-1.154391,1.012711


In [38]:
# compute the mean of each column
df2.mean()

one      0.512901
two     -0.536544
three    0.584668
dtype: float64

In [39]:
# fill the missing values in each column
for c in df.columns:
    df2[c].fillna(df[c].mean(), inplace=True)

df2

,one,two,three
a,3.060336,-2.625102,1.193727
b,0.512901,-0.536544,0.584668
c,-1.144262,1.758824,-0.205974
d,1.363693,-0.125507,0.338207
e,-1.228161,-1.154391,1.012711


# Broadcasting

Binary arithmetic operators are applied element by element to dataframes assuming equal sized dataframes.

Broadcasting refers to the fact that Python will reuse a scalar in order to complete the binary operation.


In [40]:
df = pandas.DataFrame([[1,2],[3,4]])
df

,0,1
0,1,2
1,3,4


In [41]:
# element by element operation
df + df

,0,1
0,2,4
1,6,8


In [42]:
# broadcasting a scalar
# NOTE: the scalar is applied to ALL elements
#       of the dataframe
df + 10

,0,1
0,11,12
1,13,14


In [43]:
# we can now say things like this
df + df == 2*df

,0,1
0,True,True
1,True,True


# Duplicate Data

Identify and remove duplicate rows in a DataFrame
 

In [44]:
# make a dataframe with duplicate rows 'b' and 'e'
df2.iloc[4,:] = df2.iloc[1,:]
df2

,one,two,three
a,3.060336,-2.625102,1.193727
b,0.512901,-0.536544,0.584668
c,-1.144262,1.758824,-0.205974
d,1.363693,-0.125507,0.338207
e,0.512901,-0.536544,0.584668


In [45]:
# check if there is duplication
df2.duplicated()

a    False
b    False
c    False
d    False
e     True
dtype: bool

In [46]:
# drop e!
df2.drop_duplicates()

,one,two,three
a,3.060336,-2.625102,1.193727
b,0.512901,-0.536544,0.584668
c,-1.144262,1.758824,-0.205974
d,1.363693,-0.125507,0.338207


By default **duplicated()** and **drop_duplicates()** keep the first and identify other reoccuring instances as duplicates.

# Reading

* 3.1 [Pandas](https://jakevdp.github.io/PythonDataScienceHandbook/03.01-introducing-pandas-objects.html)
* 3.2 [Data Indexing and Selection](https://jakevdp.github.io/PythonDataScienceHandbook/03.02-data-indexing-and-selection.html)
* 3.3 [Operating on Data in Pandas](https://jakevdp.github.io/PythonDataScienceHandbook/03.03-operations-in-pandas.html)
* 3.4 [Handling Missing Data](https://jakevdp.github.io/PythonDataScienceHandbook/03.04-missing-values.html)
* 3.6 [Combining Datasets: Concat and Append](https://jakevdp.github.io/PythonDataScienceHandbook/03.06-concat-and-append.html)
* 3.7 [Combining Datasets: Merge and Join](https://jakevdp.github.io/PythonDataScienceHandbook/03.07-merge-and-join.html)
* 3.8 [Aggregation and Grouping](https://jakevdp.github.io/PythonDataScienceHandbook/03.08-aggregation-and-grouping.html)